<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, input_dim, n_heads, hidden_dim, n_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=n_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src):
        output = self.transformer_encoder(src)
        output = self.fc(output)
        return output

# Example usage
model = TransformerModel(input_dim=512, n_heads=8, hidden_dim=2048, n_layers=6, output_dim=10)
src = torch.randn(10, 32, 512)  # (sequence length, batch size, input dimension)
output = model(src)
print("Transformer output shape:", output.shape)